<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


In [43]:
from openai import OpenAI
from pprint import pprint

In [2]:
client = OpenAI()

### Могут ли LLM заниматься математикой?

Если LLM просто предсказывают следующие наиболее вероятные жетоны, могут ли они на самом деле решать математические задачи? Или они будут галлюцинировать ответ?

In [3]:
answer = 123456 * 456789

In [4]:
answer

56393342784

In [5]:
response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
    {'role':'system','content':'You help solve math problems'},
    {'role':'user','content':'What is 123456 times 456789?'}
    ]

)

In [6]:
print(response.choices[0].message.content)

123456 times 456789 equals 56,365,995,584.


In [7]:
response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [
    {'role':'system','content':'You help solve math problems'},
    {'role':'user','content':'What is 123456 * 456789?'}
    ]

)

print(response.choices[0].message.content)

123456 multiplied by 456789 equals 56394002424.


### Asking for code instead, and then running that code!

In [11]:
response = client.chat.completions.create(
    model = 'gpt-3.5-turbo-0125',
    messages = [
    {'role':'system','content':'You convert word problems into Python code. Only return the python code equivalent, no other commentary.'},
    {'role':'user','content':'Return the python code for: What is 123456 times 456789?'}
    ],
    temperature = 0

)

In [13]:
response.choices[0].message.content

'result = 123456 * 456789\nprint(result)'

In [10]:
# We could then just run the code!
eval(response.choices[0].message.content)

SyntaxError: invalid syntax (<string>, line 1)

# Assistant с Code Interpreter

Очевидно, что мы можем расширить эту идею! Здесь мы показываем использование LLM для преобразования словесных задач в простые математические формулы Python, но в теории LLM может превратить любое словесное описание на естественном языке в код Python, а затем просто запустить сам код! Это ключевая идея, лежащая в основе интерпретатора кода! Ассистенты OpenAI позволяют нам подключить LLM к инструменту-интерпретатору кода. Давайте изучим это подробнее!


При создании помощника вам нужно учесть несколько параметров:

* Инструкции: Здесь вы можете определить, как должен вести себя или отвечать помощник. Эти инструкции определяют общий тон, стиль и тип ответов помощника.

* Выбор модели: Вы можете выбрать одну из различных моделей GPT-3.5 или GPT-4, включая те, которые специально настроены для выполнения конкретных задач. Если вы планируете использовать такие инструменты, как инструмент Retrieval, вам понадобятся специальные модели, например gpt-3.5-turbo-1106 или gpt-4-1106-preview.

* Интеграция с инструментами: API поддерживает интеграцию с такими инструментами, как Code Interpreter и Retrieval, которые разрабатываются и поддерживаются OpenAI. Эти инструменты расширяют возможности вашего помощника, позволяя ему интерпретировать код или извлекать информацию более эффективно.

* Пользовательские функции: Еще одна мощная особенность API - возможность определять подписи пользовательских функций. Эта функция повторяет функцию вызова функций API, позволяя вашему помощнику выполнять определенные задачи или операции в соответствии с вашими требованиями.

Давайте начнем с очень простого помощника, который имеет доступ к инструменту интерпретатора кода:

In [15]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="Вы преобразуете математические задачи в код Python, а затем запускаете код, чтобы показать ответ.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo" # Check the docs for the latest models
)

### Потоки

#### Создать Поток

Давайте разберем концепцию создания нити в контексте использования API Assistants с OpenAI, сосредоточившись на том, чтобы сделать ее более понятной и образовательной.

#### Понимание Потока

Думайте о Потоке как о цифровом разговоре или чате. В реальной жизни, когда вы начинаете разговаривать с кем-то, у вас происходит ряд обменов мнениями, которые и составляют разговор. Точно так же и в цифровом мире, когда пользователь начинает взаимодействовать с API (например, с API OpenAI Assistants), это взаимодействие группируется в то, что мы называем "поток".

#### Зачем создавать поток?

1. **Организация**: Каждый пользователь получает свою собственную "Поток", как персональный чат. Таким образом, разговоры упорядочиваются и отделяются друг от друга.
2. **Управление контекстом**:  В этой "Потоке" вы можете передавать определенную информацию или файлы, которые относятся к конкретному пользователю. Это как личная папка для беседы каждого пользователя, в которой хранятся все его данные и история в одном месте.

#### Как работаеют потоки?

- **Нет ограничений по размеру**: Вы можете представить себе Нить как бесконечный разговор. Нет предела тому, сколько вы можете в нее добавить. Вы можете продолжать разговор, добавляя столько сообщений (или взаимодействий), сколько необходимо.
- **Оптимизация разговоров**: Помощник (в данном случае модель искусственного интеллекта, с которой вы взаимодействуете) довольно умен. Он эффективно управляет беседой, используя специальные приемы, такие как усечение. Это означает, что если разговор становится слишком длинным, помощник сосредоточится на наиболее важных частях, чтобы сохранить плавность взаимодействия.
- **Упрощение сложности**: Когда вы используете эту систему, вы, по сути, передаете бразды правления помощнику. Он сам решает, какой объем информации (или сколько входных маркеров) учитывать для каждого ответа. Это облегчает вам задачу, поскольку вам не нужно беспокоиться о технических аспектах управления ходом беседы.

#### Что это значит для вас?

- **Простота использования**: Вы сосредотачиваетесь на том, что хотите спросить или сказать, а система делает все остальное. Она создана для того, чтобы поддерживать беседу, не отвлекаясь на технические детали.
- **Учет затрат**: Хотя это значительно упрощает работу, это также означает, что у вас меньше прямого контроля над потенциальными расходами, поскольку помощник сам решает, сколько данных обрабатывать при каждом взаимодействии.

В целом, создание потока с помощью API Assistants - это как начало цифрового разговора с искусственным интеллектом, в котором каждый пользователь получает свою собственную персональную чат-комнату. Такая система делает управление взаимодействием более простым и эффективным, хотя и означает, что вы не имеете прямого контроля над техническими аспектами разговора.

In [16]:
thread = client.beta.threads.create()

In [17]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is 123456 times 456789?"
)

In [18]:
message

Message(id='msg_staFFKB7enF0dfrZsIDb0MCn', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is 123456 times 456789?'), type='text')], created_at=1714546275, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_ywlT5eCcQkmWIAODlwpducDv')

In [20]:
client.beta.threads.messages.list(thread_id='thread_ywlT5eCcQkmWIAODlwpducDv')

SyncCursorPage[Message](data=[Message(id='msg_staFFKB7enF0dfrZsIDb0MCn', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is 123456 times 456789?'), type='text')], created_at=1714546275, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_ywlT5eCcQkmWIAODlwpducDv')], object='list', first_id='msg_staFFKB7enF0dfrZsIDb0MCn', last_id='msg_staFFKB7enF0dfrZsIDb0MCn', has_more=False)

## Запуск помощника

### Что означает "управлять помощником"?

Представьте себе помощника (например, ChatGPT) в виде полезного робота, готового ответить на вопросы или выполнить задания. Когда вы "запускаете помощника", вы, по сути, будите этого робота и говорите: "Эй, мне нужно кое-что для тебя сделать".

### Как это работает?

1. **Начало процесса**: Вы начинаете с создания так называемого "Запуска". Это похоже на нажатие кнопки "Пуск" на машине. Это дает команду помощнику начать обращать внимание на разговор, или "Нить", который происходит.

2. **Чтение потока**: После запуска запуска Помощник начинает читать Поток. Это как бы дает роботу историю того, что обсуждалось до сих пор, чтобы он мог понять контекст и то, что нужно ответить или сделать дальше.

3. **Реагирование на сообщения**: По мере выполнения задания помощник добавляет свои ответы в поток. Это сообщения с ролью "помощник", показывающие, что это ответы ИИ, а не пользователя.

4. **Умное управление контекстом**: Ассистент призван автоматически выбирать, какие предыдущие сообщения ему следует учитывать при ответе. Это важно, потому что он должен знать, что относится к текущему вопросу или задаче. Считайте, что помощник делает быстрые заметки о наиболее важных моментах разговора, чтобы не сбиться с пути.

### Что вы должны знать?

- **Влияние на ценообразование и производительность**: То, как помощник выбирает контекст (или решает, на каких частях разговора сосредоточиться), влияет на две вещи: сколько стоит работа помощника (поскольку обработка большого количества данных может быть дороже) и насколько хорошо помощник работает (поскольку сосредоточение на правильном контексте может привести к лучшим, более точным ответам).
- **Оптимизация и эволюция**: Весь этот процесс был отлажен на основе опыта OpenAI по созданию таких систем, как ChatGPT. Однако он постоянно развивается. Это означает, что то, как Ассистент работает сегодня, может быть улучшено или немного изменено в будущем, чтобы сделать его еще лучше.

### Дополнительные инструкции

- **Настройка запуска**: Вы можете дать Помощнику особые инструкции для каждого Запуска. Это все равно что сказать роботу: "В этот раз сосредоточься на этой конкретной задаче или ответь таким-то способом". Но помните, что эти инструкции отменяют стандартный способ реагирования Помощника, поэтому используйте эту функцию обдуманно.

В общем, "Запуск помощника" - это этап, на котором вы активируете ИИ, чтобы он начал обрабатывать и отвечать на сообщения пользователя в беседе. Он включает в себя чтение "Потоков", соответствующее реагирование и управление контекстом разговора с учетом затрат и производительности. Этот процесс постоянно развивается и может быть настроен с помощью специальных инструкций для каждого Запуска.

In [30]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Будьте очень вежливы в своем ответе и признайте, что заниматься математикой - это весело. (Отвечайте на русском языке)"
)

In [31]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [32]:
run.status

'completed'

In [33]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [36]:
messages

SyncCursorPage[Message](data=[Message(id='msg_TjM5TPzZHc8d2QmStHvr82F9', assistant_id='asst_kCHX8rGZcgXJE16aX08fQIGz', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Результат умножения 123456 на 456789 равен 56,393,342,784.\n\nЗаниматься математикой может быть весело, не так ли? 😄'), type='text')], created_at=1714549407, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_anZX7Qjk3lixOWDVAvvtyGuI', status=None, thread_id='thread_ywlT5eCcQkmWIAODlwpducDv'), Message(id='msg_xAs53H2jRdmi73Vsb3p0165d', assistant_id='asst_kCHX8rGZcgXJE16aX08fQIGz', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="The result of multiplying 123456 by 456789 is 56,393,342,784. \n\nMath can be fun, don't you think? 😄"), type='text')], created_at=1714549248, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run

In [35]:
print(messages.data[0].content[0].text.value)

Результат умножения 123456 на 456789 равен 56,393,342,784.

Заниматься математикой может быть весело, не так ли? 😄


In [37]:
messages.data

[Message(id='msg_TjM5TPzZHc8d2QmStHvr82F9', assistant_id='asst_kCHX8rGZcgXJE16aX08fQIGz', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Результат умножения 123456 на 456789 равен 56,393,342,784.\n\nЗаниматься математикой может быть весело, не так ли? 😄'), type='text')], created_at=1714549407, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_anZX7Qjk3lixOWDVAvvtyGuI', status=None, thread_id='thread_ywlT5eCcQkmWIAODlwpducDv'),
 Message(id='msg_xAs53H2jRdmi73Vsb3p0165d', assistant_id='asst_kCHX8rGZcgXJE16aX08fQIGz', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="The result of multiplying 123456 by 456789 is 56,393,342,784. \n\nMath can be fun, don't you think? 😄"), type='text')], created_at=1714549248, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Z4glJyQyuTZ8Q2nsB3b

In [38]:
# Это показывает их в порядке убывания последнего сообщения
for thread_message in messages.data:
    print(thread_message.content[0].text.value)

Результат умножения 123456 на 456789 равен 56,393,342,784.

Заниматься математикой может быть весело, не так ли? 😄
The result of multiplying 123456 by 456789 is 56,393,342,784. 

Math can be fun, don't you think? 😄
What is 123456 times 456789?


In [39]:
# Это покажет их в порядке возрастания запроса
for thread_message in messages.data[::-1]:
    print(thread_message.content[0].text.value)
    print('\n')

What is 123456 times 456789?


The result of multiplying 123456 by 456789 is 56,393,342,784. 

Math can be fun, don't you think? 😄


Результат умножения 123456 на 456789 равен 56,393,342,784.

Заниматься математикой может быть весело, не так ли? 😄




### Подробная информация о работе:

Мы можем увидеть, какие шаги предпринял помощник, чтобы создать ответ:

In [40]:
from openai import OpenAI
client = OpenAI()

run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)
print(run_steps)


SyncCursorPage[RunStep](data=[RunStep(id='step_XKXZyhd79FmB02gIBblUGctj', assistant_id='asst_kCHX8rGZcgXJE16aX08fQIGz', cancelled_at=None, completed_at=1714549408, created_at=1714549407, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_anZX7Qjk3lixOWDVAvvtyGuI', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_TjM5TPzZHc8d2QmStHvr82F9'), type='message_creation'), thread_id='thread_ywlT5eCcQkmWIAODlwpducDv', type='message_creation', usage=Usage(completion_tokens=50, prompt_tokens=234, total_tokens=284), expires_at=None)], object='list', first_id='step_XKXZyhd79FmB02gIBblUGctj', last_id='step_XKXZyhd79FmB02gIBblUGctj', has_more=False)


In [41]:
for step in run_steps:
    print(step.step_details)
    print('\n')

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_TjM5TPzZHc8d2QmStHvr82F9'), type='message_creation')




## Listing and Deleting Assistants

In [44]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
pprint(my_assistants.data)

[Assistant(id='asst_kCHX8rGZcgXJE16aX08fQIGz', created_at=1714546267, description=None, instructions='You convert math problems into Python code and then run the code to show the answer', metadata={}, model='gpt-3.5-turbo', name='Math Tutor', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0),
 Assistant(id='asst_aysepRQBwzvxUtnvJjbkHdMh', created_at=1714205542, description=None, instructions='You are an expert in the field of astrological counseling. Use your knowledge base to answer questions related to astrology and its explanations.', metadata={}, model='gpt-4-turbo-2024-04-09', name="You're an astrological assistant", object='assistant', tools=[FileSearchTool(type='file_search')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFile

In [68]:
response = client.beta.assistants.delete('asst_1WSNjWGUgE6PezyHQNKkO9nT')
print(response)

AssistantDeleted(id='asst_1WSNjWGUgE6PezyHQNKkO9nT', deleted=True, object='assistant.deleted')


In [46]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
pprint(my_assistants.data)

[Assistant(id='asst_kCHX8rGZcgXJE16aX08fQIGz', created_at=1714546267, description=None, instructions='You convert math problems into Python code and then run the code to show the answer', metadata={}, model='gpt-3.5-turbo', name='Math Tutor', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0),
 Assistant(id='asst_aysepRQBwzvxUtnvJjbkHdMh', created_at=1714205542, description=None, instructions='You are an expert in the field of astrological counseling. Use your knowledge base to answer questions related to astrology and its explanations.', metadata={}, model='gpt-4-turbo-2024-04-09', name="You're an astrological assistant", object='assistant', tools=[FileSearchTool(type='file_search')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFile